In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
pd.set_option("display.max_columns",None) #Afficer toutes les colonnes pour un dataset long
# Charger le jeu de données
data = pd.read_excel("DATA_EX.xlsx")

data.head()

,station_id,datetime,temp_max,temp_min,temp,precip,humidity,pressure,wiEX_speed,wiEX_direction
0,REX0000001,1982-01-01,35.46,15.39,24.50,0.0,20.06,96.71,4.02,31.44
1,REX0000001,1982-01-02,34.97,14.98,25.16,0.0,19.75,96.59,3.14,35.69
2,REX0000001,1982-01-03,35.73,18.58,26.32,0.0,21.38,96.58,3.53,41.69
3,REX0000001,1982-01-04,35.70,18.12,26.82,0.0,21.50,96.63,3.46,96.62
4,REX0000001,1982-01-05,35.01,17.71,26.26,0.0,23.81,96.60,3.13,135.12


In [5]:
# Création des jeux de données d'entraînement et de test
train_size = int(len(data) * 0.8)  # 80% pour l'entraînement
train_data = data[:train_size]
test_data = data[train_size:]

train_data = train_data.drop(['station_id','datetime'], axis=1)
test_data = test_data.drop(['station_id','datetime'], axis=1)


# Normalisation des données
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Création de la fonction pour créer des séquences
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Choix du nombre de pas de temps à prendre en compte
TIME_STEPS = 24

# Préparation des données pour l'entraînement et le test
X_train, y_train = create_sequences(train_data[:, :-1], train_data[:, -1], TIME_STEPS)
X_test, y_test = create_sequences(test_data[:, :-1], test_data[:, -1], TIME_STEPS)

# Création du modèle LSTM
from keras.models import Sequential
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dense(1))

# Compilation du modèle
model.compile(optimizer='adam', loss='mse')

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=1, shuffle=False)

# Visualisation de l'historique de l'entraînement
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

# Prédiction sur l'année suivante
X_pred = np.array(data[-TIME_STEPS:, :-1])
X_pred = np.reshape(X_pred, (1, X_pred.shape[0], X_pred.shape[1]))
prediction = model.predict(X_pred)
prediction = scaler.inverse_transform(np.hstack((X_pred[0], prediction)))
prediction = prediction[:, -1]

# Affichage de la prédiction
print('Prédiction pour l\'année suivante:')
print(prediction)


NameError: name 'LSTM' is not defined